# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("../WeatherPy/output_data/cities.csv")
data.head()

,Unnamed: 0,City,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,-73.9662,42.6001,30.99,63,93,15.01,US,1611770884
1,1,Hilo,-155.0900,19.7297,69.80,83,90,3.44,US,1611770815
2,2,Yarmouth,-70.2286,41.7057,39.99,87,90,9.22,US,1611771075
3,3,Chara,118.2631,56.9064,-28.68,77,100,0.78,RU,1611771076
4,4,San Miguelito,-79.5000,9.0333,89.60,62,40,16.11,PA,1611771076


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
marker_locations = data[["Lat", "Lng"]]
humidity=data["Humidity"]
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=5)
fig=gmaps.figure()
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [4]:
fit_weather=data.loc[(data["Max Temp"]>75) & (data["Max Temp"]<85) & (data["Wind Speed"]<10) & (data["Cloudiness"]==0)]
fit_weather

,Unnamed: 0,City,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,San Martín,-68.4681,-33.0810,84.20,24,0,9.22,AR,1611771087
122,122,Pisco,-76.2167,-13.7000,75.20,64,0,8.05,PE,1611770748
124,124,São Filipe,-24.4956,14.8961,76.93,63,0,5.59,CV,1611770786
256,256,Saint-Marc,-72.6938,19.1082,82.26,52,0,6.78,HT,1611771123
452,452,Emerald,148.1667,-23.5333,77.00,73,0,8.05,AU,1611771164
467,467,Dzaoudzi,45.2699,-12.7887,82.40,78,0,5.75,YT,1611771167
504,504,Thanatpin,96.3033,17.2119,75.20,83,0,3.44,MM,1611770953
513,513,Acajutla,-89.8275,13.5928,84.20,54,0,6.91,SV,1611771153
553,553,Valparaíso,-71.6273,-33.0393,77.00,44,0,6.91,CL,1611770998


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Drop null values
hotel_df = fit_weather.dropna()
# Add Hotel Name column
hotel_df["Hotel Name"] = ""
# Set parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# Loop over rows in dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make API request and print url
    hotel_name = requests.get(base_url, params=params)

    # Convert to json
    hotel_name = hotel_name.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,Unnamed: 0,City,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
70,70,San Martín,-68.4681,-33.0810,84.20,24,0,9.22,AR,1611771087,Finca Vivencias
122,122,Pisco,-76.2167,-13.7000,75.20,64,0,8.05,PE,1611770748,Hotel San Isidro Oficial
124,124,São Filipe,-24.4956,14.8961,76.93,63,0,5.59,CV,1611770786,Tortuga B&B
256,256,Saint-Marc,-72.6938,19.1082,82.26,52,0,6.78,HT,1611771123,Maguana Hotel
452,452,Emerald,148.1667,-23.5333,77.00,73,0,8.05,AU,1611771164,The Irish Village
467,467,Dzaoudzi,45.2699,-12.7887,82.40,78,0,5.75,YT,1611771167,Le Rocher
504,504,Thanatpin,96.3033,17.2119,75.20,83,0,3.44,MM,1611770953,Lo Ta Ya Garden
513,513,Acajutla,-89.8275,13.5928,84.20,54,0,6.91,SV,1611771153,asi es
553,553,Valparaíso,-71.6273,-33.0393,77.00,44,0,6.91,CL,1611770998,Hotel Ultramar


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
hotel_df_coords = hotel_df[["Lat","Lng"]]
hotel_layer = gmaps.marker_layer(hotel_df_coords, info_box_content = hotel_info)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))